In [2]:
from bs4 import BeautifulSoup
import requests as req
import lxml
import pandas as pd
import numpy as np

Рассмотрим пример работы с каким-то конкретным магазином. Например: ONEVAN S925 STORE (https://prnt.sc/u1k9ya) 

In [5]:
 page = req.get('https://aliexpress.ru/store/top-rated-products/5729146.html')
 bs = BeautifulSoup(page.content, 'lxml')

In [9]:
items = bs.find_all('li', attrs = {'item'})
len(items)

44

Получили карточки товара (данные ещё надо чистить, они с дубликатами, но как факт, парсинг возможен). Раз html код позволяет обращаться к ним, то мы спокойно сможем вытащить оттуда:
- ссылку на продукт
- название продукта
- число заказов
- рейтинг
- цену

In [11]:
# пример отображения одного товара
items[0]

<li class="item">
<div class="img">
<div class="pic">
<a class="pic-rind" href="//aliexpress.ru/item/4001055185417.html"><img alt="Мода 925 стерлингового серебра с О-образным вырезом цепи ожерелье 0,3 см/0,4 см/0,5 см циркон ожерелье для женщин подарок, летнее, модное, ювелирно..." class="picCore lazy-load" image-src="//ae01.alicdn.com/kf/Hed1608cf2e0d4bacb765bb58738ce0daf/Trendy-925-Sterling-Silver-O-Chain-Necklace-0-3cm-0-4cm-0-5cm-Zircon-Necklace-For.jpg_200x200.jpg"/></a>
</div>
<div class="discount">
<span class="rate">35</span>
</div>
</div>
<div class="detail">
<h3>
<a href="//aliexpress.ru/item/4001055185417.html" title="Мода 925 стерлингового серебра с О-образным вырезом цепи ожерелье 0,3 см/0,4 см/0,5 см циркон ожерелье для женщин подарок, летнее, модное, ювелирно...">Мода 925 стерлингового серебра с О-образным вырезом цепи...</a>
</h3>
<div class="cost"><b>180,05 руб.</b> / шт.</div>
<div class="cost-old"><del>276,88 руб. / шт.</del></div>
<div class="recent-order"> Заказов(

Соответственно мы можем с интервалом, например, раз в сутки (можно и чаще, чтобы чувствительнее ловить изменения — например, 6/12 часов) возвращаться за информацией о числе заказов. И смотреть динамику. <br>
<br>
Имеет резон смотреть на динамику процентного изменения числа заказов, чтобы нормализовать измерения.

Таким образом, мы получаем перспективные товары в рамках одного магазина. 

<h3>Вопрос с конкуренцией</h3>

Так как мы забрали ссылки на конкретные товары, то мы можем перейти на их страницы, например на [конкретную подвеску](https://aliexpress.ru/item/4001055185417.html?spm=a2g0v.12010610.8148356.1.feb1190cj5jTUJ) и забрать тип товара, в котором она находится, из [характеристик](https://prnt.sc/u1kgd3). 

Сейчас у меня не вышло это сделать для примера (блок Характеристик открывается после работы js, пока не знаю, как это спарсить, не работал с таким, но уверен, что это возможно). 

В дальнейшем мы будем смотреть на количество товаров категории. И выбирать, соответствествнно, категории, где товаров не так много. <br>
<br>
Дополнительно можно также парсить и суммировать число продаж популярных товаров в категории, чтобы также смотреть на их динамику. 
<br><br>
Популярная по процентной динамике категория с малым количеством товара + товар с хорошей динамикой - идеальный вариант.

<h3>Метрика</h3>

Кастомной метрикой для измерения потецниала товара может служить следующий коэффициент: 

$$coef\_dyn = \frac{item\_dyn}{mean\_top\_dyn},$$ где в знаменателе динамика рассматриваемого продукта, в знаменателе - средняя динамика по топу в категории. <br>
Чем выше данный коэффициент, тем интереснее нам продукт. <br>
<br>
Смотреть его имеет смысл в связке с количественным приростом заказов за сутки хотя бы по топовым товарам категории (чтобы понимать её посещаемость) и с числом товаров в категории. 

<h3>Магазины</h3>

В начале мы взяли случайный магазин. <br>
Магазины для парсинга для конкретного клиента могут собираться вручную: мы ведь не хотим сделать мусорный сервис-агрегатор, где есть всё подряд, а желаем скорее действовать в рамках какой-то категории, которую обозначает заинтересованный клиент. <br>
<br>
Либо мы также можем парсить ссылки на магазины с карточки товара: они доступны в числе [прочих элементов](https://prnt.sc/u1kwan). И далее идти по уже полученному списку магазинов в нужной категории. 

<h3>P.S.</h3>

Это было моё первое посещение AliExpress. Пока достаточно чужеродный и малознакомый для меня ресурс.<br> 
Возможно, мы можем найти какие-то сервисы, которые предоставляют информацию о количество заказов по категориям с AE (я таковых не нашёл), возможно, мы можем без лишних техических затрат лучше получать информацию о востребованности категории (количестве заказов). <br><br>
Но считаю, что предложенный мной вариант вполне может быть baseline-решением данной задачи. уверен, ещё большее погружение непосредственно в ходе работы позволило бы «нарастить» данное решение, оснастив его новыми возможностями. 